In [21]:
import numpy as np
import pandas as pd
import random as rand
import requests
import json
import re
from json import loads

import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import math
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime

In [30]:
class after_settlement():
    
    def __init__(self,src,dst,order,total_pay):
        self.src = src
        self.dst = dst
        self.order = order
        self.total_pay = total_pay

        src = self.src
        dst = self.dst
        order = self.order
        total_pay = self.total_pay

        src.extend(order)
        routing = src

        loc_info = {i :[src[i]] for i in range(len(src))}
        self.loc_info = loc_info
        
        # 출발지+도착지 좌표 POI
        for i in range(len(loc_info)):
            url = f"https://apis.openapi.sk.com/tmap/pois?version=1&searchKeyword={loc_info[i]}&searchType=all" \
                f"&searchtypCd=A&reqCoordType=WGS84GEO&resCoordType=WGS84GEO&page=1&count=1&multiPoint=N&poiGroupYn=N"

            headers = {
                "Accept": "application/json",
                "appKey": "l7xxcdd1a30a6b34450a881f8083994a8cd4"
            }

            response = requests.get(url, headers=headers)
            jsonObj = json.loads(response.text)
            loc_info[i].append(float(jsonObj['searchPoiInfo']['pois']['poi'][0]['frontLon']))
            loc_info[i].append(float(jsonObj['searchPoiInfo']['pois']['poi'][0]['frontLat']))
        
        addr_info = {loc_info[i][0] : {'no': i, 'lon': loc_info[i][1], 'lat': loc_info[i][2]} for i in range(len(loc_info))}
        passenger_routing = routing
        self.passenger_routing = passenger_routing

        ### move alone 시간, 요금 계산
        individual_distance = []
        individual_duration = []
        individual_cost = []

        url = "https://apis.openapi.sk.com/tmap/routes?version=1&callback=function"

        for i in range(len(loc_info)-1):
            payload = {
                "tollgateFareOption": 16,
                "roadType": 32,
                "directionOption": 1,
                
                # j번째 승객의 하차지역
                "endX": addr_info[passenger_routing[i+1]]['lon'],
                "endY": addr_info[passenger_routing[i+1]]['lat'],
                ##
                "endRpFlag": "G",
                "reqCoordType": "WGS84GEO",        
                # 출발지역
                "startX": addr_info[passenger_routing[0]]['lon'],
                "startY": addr_info[passenger_routing[0]]['lat'],
                ##
                "gpsTime": datetime.now().strftime('%Y%m%d%H%M%S'), #YYYYMMDDhhmmss
                # "speed": 24, # 서울시 자동차 통행 속도
                "uncetaintyP": 1,
                "uncetaintyA": 1,
                "uncetaintyAP": 1,
                "carType": 0,
                "detailPosFlag": "2",
                "resCoordType": "WGS84GEO",
                "sort": "index",
                "searchOption": 0,
                "totalValue": 2,
                "trafficInfo": "Y", # 현재교통상황 반영
                "mainRoadInfo": "N"
            }
            headers = {
                "accept": "application/json",
                "content-type": "application/json",
                "appKey": "l7xx4d23b06a733d4f2e9c5365ca49c3431a"
            }
            response = requests.post(url, json=payload, headers=headers)
            jsonObj = json.loads(response.text)
            individual_distance.append(jsonObj['features'][0]['properties']['totalDistance'])
            individual_duration.append(jsonObj['features'][0]['properties']['totalTime'])
            individual_cost.append(jsonObj['features'][0]['properties']['taxiFare'])

        self.individual_distance = individual_distance
        self.individual_duration = individual_duration
        self.individual_cost = individual_cost

        ### move together 시간, 요금 계산
        routing_distance = []
        routing_duration = []
        routing_cost = []

        url = "https://apis.openapi.sk.com/tmap/routes?version=1&callback=function"

        for j in range(1,len(loc_info)):
            # print('start')
            passlist = ''
            # 내리는 순서에 따라 경유지 들리는 횟수가 달라진다. >> 경유지 설정
            for i in range(1,j+1):
                if i==j:
                    break
                passlist += str(addr_info[passenger_routing[i]]['lon'])+','+str(addr_info[passenger_routing[i]]['lat'])
                if i!=j-1:
                    passlist += '_'
            ###
            payload = {
                "tollgateFareOption": 16,
                "roadType": 32,
                "directionOption": 1,

                # j번째 승객의 하차지역
                "endX": addr_info[passenger_routing[j]]['lon'],
                "endY": addr_info[passenger_routing[j]]['lat'],
                ###
                "endRpFlag": "G",
                "reqCoordType": "WGS84GEO",
                # 출발지역
                "startX": addr_info[passenger_routing[0]]['lon'],
                "startY": addr_info[passenger_routing[0]]['lat'],
                ###
                "gpsTime": datetime.now().strftime('%Y%m%d%H%M%S'),
                # "speed": 24,
                "uncetaintyP": 1,
                "uncetaintyA": 1,
                "uncetaintyAP": 1,
                "carType": 0,
                "detailPosFlag": "2",
                "resCoordType": "WGS84GEO",
                "sort": "index",
                "mainRoadInfo": "N",
                "trafficInfo": "Y", # 현재교통상황 반영
                "totalValue": 2
            }
            # passlist 없다면 pass
            if passlist!='':
                payload["passList"] = passlist
                ##
                headers = {
                    "accept": "application/json",
                    "content-type": "application/json",
                    "appKey": "l7xxcdd1a30a6b34450a881f8083994a8cd4"
                }
                # print(passlist)
                response = requests.post(url, json=payload, headers=headers)
                jsonObj = json.loads(response.text)

                routing_distance.append(jsonObj['features'][0]['properties']['totalDistance'])
                routing_duration.append(jsonObj['features'][0]['properties']['totalTime'])
                routing_cost.append(jsonObj['features'][0]['properties']['taxiFare'])
            else: pass
            # print('end')

        # 첫번째 하차 승객 정보 추가
        routing_distance.insert(0,individual_distance[0])
        routing_duration.insert(0,individual_duration[0])
        routing_cost.insert(0,individual_cost[0]) 
        # 최종요금 보정
        routing_cost = [int(x/max(routing_cost)*total_pay) for x in routing_cost]

        self.routing_distance = routing_distance
        self.routing_duration = routing_duration
        self.routing_cost = routing_cost

        # 지연 비율 >> 요금 정산에 필요
        delay_ratio = [(routing_duration[i]-individual_duration[i])/individual_duration[i] for i in range(len(loc_info)-1)]

        self.delay_ratio = delay_ratio


    def settlement_optimizer(self):


        LB = 0.4
        UB = 0.8

        individual_cost = self.individual_cost
        delay_ratio = self.delay_ratio
        routing_cost = self.routing_cost

        T = routing_cost[-1]

        I1, I2, I3 = individual_cost
        R1, R2, R3 = delay_ratio

        ## matching ALGorithm
        try:
          # Create a new model
          m = gp.Model('cost_optimization1')
          m.Params.LogToConsole = 0

          # Create variables
          alpha = m.addVar(vtype=GRB.CONTINUOUS, name='alpha')
          beta = m.addVar(vtype=GRB.CONTINUOUS, name='beta')

          # Set objective
          m.setObjective(I1*(1-(alpha+beta*R1))+I2*(1-(alpha+beta*R2))+I3*(1-(alpha+beta*R3))-T, GRB.MINIMIZE)

          # Add constraint
          m.addConstr(max(R1,R2,R3)*beta-UB*alpha <= 0,'const_1')
          m.addConstr(max(R1,R2,R3)*beta-LB*alpha >= 0,'const_2')
          m.addConstr(I1*(1-(alpha+beta*R1))+I2*(1-(alpha+beta*R2))+I3*(1-(alpha+beta*R3))-T >= 0,'const_3')

          m.addConstr(alpha <= 0.5)
          m.addConstr(alpha >= 0)
          m.addConstr(beta <= 0.5)
          m.addConstr(beta >= 0)

          # Optimize model
          m.optimize()

          for v in m.getVars():
            globals()[v.varName] = v.x

        except gp.GurobiError as e:
          print('Error code ' + str(e.errno) + ': ' + str(e))

        self.alpha = alpha
        self.beta = beta
    
    def result(self):
        loc_info = self.loc_info

        individual_distance = self.individual_distance
        individual_duration = self.individual_duration
        individual_cost = self.individual_cost
        passenger_routing = self.passenger_routing

        routing_distance = self.routing_distance
        routing_duration = self.routing_duration
        routing_cost = self.routing_cost
        delay_ratio = self.delay_ratio

        routing_individual_cost = [int((1-alpha-beta*delay_ratio[i])*individual_cost[i]) for i in range(len(loc_info)-1)]
        beta_prime = [beta*delay_ratio[i] for i in range(len(loc_info)-1)]

        print()
        for i in range(len(loc_info)-1):
            print(f'{passenger_routing[i+1]} : {alpha*100:.2f}+{beta*delay_ratio[i]*100:.2f} => {(alpha+beta_prime[i])*100:.2f}% 할인')
        print()
        for i in range(len(loc_info)-1):
            print(f'{passenger_routing[i+1]} : {individual_cost[i]} 원 >>> {routing_individual_cost[i]} 원')
        return {'passenger':passenger_routing, 'move_alone':individual_cost, 'move_together':routing_individual_cost,'paramter':{'alpha':alpha,'beta_prime':beta_prime}}

In [31]:
# 사후정산 모듈
src = ['서울역']
dst = ['노원역','안암역','한양대역']
order = ['한양대역','안암역','노원역']
total_pay = 34200
# time = "2023-07-06T23:30:00+0900"

set = after_settlement(src,dst,order,total_pay)
set.settlement_optimizer()
output = set.result()


한양대역 : 34.28+0.00 => 34.28% 할인
안암역 : 34.28+9.41 => 43.69% 할인
노원역 : 34.28+13.71 => 47.99% 할인

한양대역 : 15800 원 >>> 10384 원
안암역 : 15600 원 >>> 8784 원
노원역 : 28900 원 >>> 15031 원


In [32]:
output

{'passenger': ['서울역', '한양대역', '안암역', '노원역'],
 'move_alone': [15800, 15600, 28900],
 'move_together': [10384, 8784, 15031],
 'paramter': {'alpha': 0.3427758875811067,
  'beta_prime': [0.0, 0.09411055887318405, 0.1371103550324427]}}